In [32]:
library(astsa)
library(forecast)
library(FitAR)
library(Metrics)
library(pracma)

df <- read.csv("projectdata_covid.csv")

# 1 Spectral Density and Periodigram

In [2]:
localMaxima <- function(x) {
    'Find indecies of local maxima of a sequential list. 

    Parameters
    ----------
    first: list of floats
        sequential y-vals of curve

    Returns
    -------
    list
        indecies of local maxima
    '
    
    y <- diff(c(-Inf, x)) > 0L
    rle(y)$lengths
    y <- cumsum(rle(y)$lengths)
    y <- y[seq.int(1L, length(y), 2L)]
    if (x[[1]] == x[[2]]) {
        y <- y[-1]
    }
    y
}

pgram <- function(x){
    'Discrete periodigram for a time series via FFT. 
    index/length(x) gives the sinusoid frequencies 
    from the fourier frequencies. 

    Parameters
    ----------
    first: list of floats
        time series

    Returns
    -------
    list
        periodigram of time series and list of periodigram values.
    '
    m = floor(length(x)/2)
    pgram = abs(fft(x)[2:(m+1)])^2/length(x)
    plot(pgram, type = "h")
    abline(h=0)
    return(pgram)
}

specMax <- function(x) {
    'Compute the spectral density of a time series x. 
    Return frequencies of spectral maxima in descending order.
    Recall that spectral frequencies are the same as sinusoid frequencies. 

    Parameters
    ----------
    first: list of floats
        time series data

    Returns
    -------
    list
        frequencies of spectral maxima
    '

    s <- mvspec(x)
    maxs.index <- localMaxima(s$spec)  # find indecies of local maxima
    maxs <- s$freq[maxs.index]  # frequency values of local maxima
    maxs.val <- s$spec[maxs.index]  # spec values of local maxima
    
    rv <- maxs[order(-maxs.val)]  # put maxs in decreasing order
    
    return(rv)
}

## 1.1 Duality between Spectral Density and Periodigram

In [ ]:
# Duality between spectral density and periodigram
Y.fromPeriod <- pgram(df$New.Cases)
maxFreqs.fromSpec <- specMax(df$New.Cases)

l <- length(df$New.Cases)
maxFreqs.fromPeriod <- localMaxima(Y.fromPeriod)
maxY.fromPeriod <- Y.fromPeriod[maxsIndex.fromPeriod]
maxFreqs.fromPeriod <- maxFreqs.fromPeriod[order(-maxY.fromPeriod)]/l

maxFreqs.fromPeriod
maxFreqs.fromSpec

Overlap between the largest spike in the spectral and periodigram domains

In [66]:
sarima2 <- function(df, p=0,d=0,q=0, P=0,D=0,Q=0,S=1, plot = TRUE) {
    
    model <- sarima(df, p,d,q, P,D,Q,S, details=FALSE)
    
    # non-seasonal difference:
    while (d > 0){
        df <- diff(df)
        d <- d-1
    }
    # seasonal difference:
    while (D > 0){
        df <- diff(df, lag = S)
        D <- D-1
    }
    
    ARMAcoefficients <- list("AR" = c(), "MA" = c())
    ARpoly <- list("little" = c(1), "big" = c(1, rep(0, (S-1))))
    MApoly <- list("little" = c(1), "big" = c(1, rep(0, (S-1))))
    
    # non-seasonal coefficient polynomial. 
    # Polynomials for "little phis" and "little thetas":
    if(p > 0){
        for(i in c(1:p)){
            ARpoly$little <- c(ARpoly$little, model$ttable[i])
        }
    }
    if(q > 0){
        for(i in c(1:q)){
            MApoly$little <- c(MApoly$little, model$ttable[i+p])
        }
    }
    
    # seasonal coefficient polynomial. 
    # Polynomials for "big Phis" and "Big Thetas":
    if(Q > 0){
        for(i in c(1:(P-1))){
            ARpoly$big <- c(ARpoly$big, model$ttable[i+p+q], rep(0, (S-1)))
        }
        ARpoly$big <- c(ARpoly$big, model$ttable[p+q+P])
    }
    if(P > 0){
        for(i in c(1:(Q-1))){
            MApoly$big <- c(MApoly$big, model$ttable[i+p+q+P], rep(0, (S-1)))
        }
        ARpoly$big <- c(ARpoly$big, model$ttable[p+q+P+Q])
    }
    
    # Full ARMA coefficients:
    ARcoef <- polymul(rev(ARpoly$little), rev(ARpoly$big))
    MAcoef <- polymul(rev(MApoly$little), rev(MApoly$big))
    
    
    
    acfs <- acf2(df, max.lag = (length(df)/2), plot = plot)
    
    
}
    


In [92]:
polymul(c(1,2,30,4,5), c(1,0))

[1]  1  2 30  4  5  0

In [43]:
m1 <- sarima(df$New.Cases, p=3,d=0,q=2, P=4,D=0,Q=1,S=7, details = FALSE)

In [89]:
l <- list("one" = c(1,10,3,5), "two" =c())
l$one <- rev(l$one)
l$one
polymul(c(1,0,1),c(1,1,1))

[1]  5  3 10  1

[1] 1 1 2 1 1

In [74]:
names(m1)
m1$ttable[7]; m1$ttable

[1] "fit"                "degrees_of_freedom" "ttable"            
[4] "AIC"                "AICc"               "BIC"

[1] 0.4543

,Estimate,SE,t.value,p.value
ar1,-0.0229,0.2717,-0.0842,0.9332
ar2,-0.4751,0.1415,-3.3573,0.0015
ar3,0.6430,0.1132,5.6820,0.0000
ma1,0.6827,0.3225,2.1170,0.0394
ma2,0.8208,0.3588,2.2874,0.0265
sar1,-0.2237,0.5396,-0.4146,0.6803
sar2,0.4543,0.2810,1.6165,0.1124
sar3,0.1871,0.2585,0.7239,0.4726
sar4,0.2702,0.1941,1.3919,0.1702
sma1,0.6302,0.5863,1.0749,0.2877
